In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
path = "/Users/saksh.menon/Documents/GitHub/C-RNN-approach/Labels/DATASET.txt"
# path = "/home/sakshmeno/Documents/GitHub/C-RNN-approach/Labels/DATASET.txt"

In [2]:
def code_preprocessing(file):
    with open(file) as dataset_obj:
        codeLines = dataset_obj.read()
    comment_lines = []
    raw_codeLines = codeLines.replace("\t","").split("\n")
    multi_line_flag = 0
    for line_number in range(len(raw_codeLines)):
        if multi_line_flag:
            if raw_codeLines[line_number].__contains__("*/"):
                multi_line_flag = 0
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("/*"):
            if raw_codeLines[line_number].startswith("/*") and not(raw_codeLines[line_number].__contains__("*/")):
                comment_lines.append(line_number)
                multi_line_flag = 1
            elif raw_codeLines[line_number].__contains__("/*") and not(raw_codeLines[line_number].startswith("/*")):
                if raw_codeLines[line_number].__contains__("*/"):
                    psuedo_multi_line_start = raw_codeLines[line_number].find("/*")
                    psuedo_multi_line_end = raw_codeLines[line_number].find("*/")
                    temporary_line = raw_codeLines[line_number][:psuedo_multi_line_start] + raw_codeLines[line_number][psuedo_multi_line_end+2:]
                    raw_codeLines[line_number] = temporary_line

        elif raw_codeLines[line_number].startswith("//"):
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("//"):
            comment_start = raw_codeLines[line_number].find('//')
            raw_codeLines[line_number] = raw_codeLines[line_number][:comment_start]
            
    comment_lines.reverse()
    for i in comment_lines:
        raw_codeLines.pop(i)

    def insert_space(string, index):
        string_copy = ""
        for i in range(len(string)):
            if i==(index):
                string_copy += " "
                string_copy += string[i]
                string_copy += " "
                continue
            string_copy += string[i]
        return string_copy

    def find_char_indices(input_string, char):
        indices = []
        replacement_token = 0
        for index, character in enumerate(input_string):
            if character == char:
                indices.append(index + 2*replacement_token)
                replacement_token+=1
        return indices

    def space_out(string, char):
        indices = find_char_indices(string, char)

        for i in indices:
            string = insert_space(string, i)
        return string

    for line_number in range(len(raw_codeLines)):
        placeHolder = raw_codeLines[line_number]
        placeHolder = space_out(placeHolder, ";")
        placeHolder = space_out(placeHolder, "(")
        placeHolder = space_out(placeHolder, ")")
        placeHolder = space_out(placeHolder, ",")
        if placeHolder.endswith(";"):
            raw_codeLines[line_number] = "<start> " + placeHolder.replace(";","<end>")
        elif not(placeHolder.endswith(";")):
            raw_codeLines[line_number] = "<start> " + placeHolder + " <end>"
        raw_codeLines[line_number] = placeHolder

    return raw_codeLines


In [3]:
codeLines = code_preprocessing(path)

In [4]:
codeLines

['',
 '',
 '#include <Compiler.h>',
 '#include <TimeDelay.h>',
 '#include "HardwareProfile.h"',
 '#include "Peripherals/Nand.h"',
 '#include "Peripherals/Accel.h"',
 '#ifdef USE_GYRO',
 '#include "Peripherals/Gyro.h"',
 '#endif',
 '#include "Peripherals/Rtc.h"',
 '#include "USB/USB.h"',
 '#include "USB/usb_function_msd.h"',
 '#include "USB/usb_function_cdc.h"',
 '#include "MDD File System/FSIO.h"',
 '#include "Usb/USB_CDC_MSD.h"',
 '#include "Ftl/FsFtl.h"',
 '#include "Utils/FsUtils.h"',
 '#include "Utils/Fifo.h"',
 '#include "Utils/Util.h"',
 '#include "Peripherals/Analog.h"',
 '#include "Settings.h"',
 '#include "Logger.h"',
 '',
 '#include "AX3 SelfTest.h"',
 '',
 '',
 'void RunAttached ( void )  ; ',
 'void RunLogging ( void )  ; ',
 'void TimedTasks ( void )  ; ',
 'void LedTasks ( void )  ; ',
 'void RunTestSequence ( void )  ; ',
 '',
 'void __attribute__ (  ( interrupt , auto_psv )  )  _DefaultInterrupt ( void ) ',
 '{',
 ' static unsigned int INTCON1val ; ',
 'LED_SET ( LED_MA

In [6]:
vectorizer = TfidfVectorizer()
transformed_output = vectorizer.fit_transform(codeLines)
print(vectorizer.vocabulary_)

{'include': 1044, 'compiler': 495, 'timedelay': 1958, 'hardwareprofile': 976, 'peripherals': 1517, 'nand': 1361, 'accel': 232, 'ifdef': 1031, 'use_gyro': 2049, 'gyro': 959, 'endif': 735, 'rtc': 1715, 'usb': 2032, 'usb_function_msd': 2036, 'usb_function_cdc': 2035, 'mdd': 1310, 'file': 822, 'system': 1901, 'fsio': 888, 'usb_cdc_msd': 2034, 'ftl': 890, 'fsftl': 883, 'utils': 2065, 'fsutils': 889, 'fifo': 820, 'util': 2064, 'analog': 297, 'settings': 1788, 'logger': 1259, 'ax3': 375, 'selftest': 1763, 'void': 2136, 'runattached': 1732, 'runlogging': 1733, 'timedtasks': 1959, 'ledtasks': 1225, 'runtestsequence': 1734, '__attribute__': 201, 'interrupt': 1082, 'auto_psv': 371, '_defaultinterrupt': 216, 'static': 1845, 'unsigned': 2014, 'int': 1075, 'intcon1val': 1080, 'led_set': 1221, 'led_magenta': 1217, 'intcon1': 1079, 'nop': 1391, 'reset': 1642, '_rtccinterrupt': 223, 'rtcswwdtincrement': 1722, 'rtctasks': 1724, '_int1interrupt': 219, 'loggeracceltasks': 1260, '_int2interrupt': 220, '_in

In [9]:
(vectorizer.get_feature_names_out())

array(['00', '0000', '0000001', ..., 'your_data', 'zatvoren', 'zero'],
      dtype=object)

In [10]:
all_feature_names = vectorizer.get_feature_names_out()

for word in all_feature_names:
    
    #let's get the index in the vocabulary
    indx = vectorizer.vocabulary_.get(word)
    
    #get the score
    idf_score = vectorizer.idf_[indx]
    
    print(f"{word} : {idf_score}")

00 : 8.883634802267949
0000 : 8.883634802267949
0000001 : 8.883634802267949
00_resource_generation : 8.883634802267949
01 : 8.883634802267949
02110 : 8.883634802267949
02x : 7.630871833772582
04 : 8.883634802267949
08x : 7.274196889833849
0b00001001 : 8.883634802267949
0d100 : 8.883634802267949
0x : 7.274196889833849
0x0 : 5.465908118654584
0x00 : 7.091875333039894
0x0000 : 7.630871833772582
0x0000000f : 8.478169694159785
0x001f : 8.883634802267949
0x00ff : 7.178886710029524
0x01 : 8.190487621708005
0x02 : 8.883634802267949
0x05 : 8.883634802267949
0x06 : 8.883634802267949
0x0640 : 8.883634802267949
0x07e0 : 8.883634802267949
0x07ff : 8.883634802267949
0x0b : 8.883634802267949
0x0d : 8.883634802267949
0x0e : 8.883634802267949
0x0f : 6.743568638771679
0x1 : 7.379557405491676
0x10 : 8.478169694159785
0x10000ul : 8.883634802267949
0x11 : 8.883634802267949
0x113 : 8.883634802267949
0x12 : 8.883634802267949
0x1234 : 8.883634802267949
0x12345678 : 8.883634802267949
0x13 : 8.883634802267949
0

ftl : 8.883634802267949
ftlflush : 8.478169694159785
ftlincrementinactivity : 8.883634802267949
ftlshutdown : 8.478169694159785
ftlstartup : 8.883634802267949
fullest : 8.883634802267949
fully : 8.883634802267949
funds : 7.78502251359984
funds_timeout : 8.883634802267949
fw : 8.883634802267949
fwrite : 8.883634802267949
g_assert : 8.478169694159785
g_autoptr : 8.883634802267949
g_define_autoptr_cleanup_func : 8.883634802267949
g_error : 8.478169694159785
g_file_error : 8.478169694159785
g_file_error_from_errno : 8.478169694159785
g_fopen : 8.478169694159785
g_propagate_prefixed_error : 8.883634802267949
g_quark_from_static_string : 8.883634802267949
g_return_if_fail : 8.883634802267949
g_return_val_if_fail : 7.630871833772582
g_set_error : 7.78502251359984
g_strerror : 8.478169694159785
gboolean : 7.78502251359984
gchar : 8.190487621708005
gen_key_other_data : 8.190487621708005
gen_key_params : 7.178886710029524
general : 8.190487621708005
genkey_mode_pubkey_digest : 8.883634802267949


In [18]:
# for i in (transformed_output.toarray()[2]):
#     print(i)
out = vectorizer.fit_transform(list(codeLines[2]))
print(out.toarray())

ValueError: empty vocabulary; perhaps the documents only contain stop words